In [28]:
import pandas as pd
import numpy as np

from datetime import datetime as dt
from datetime import timedelta

from functools import reduce


In [29]:
connection = pd.read_csv('../data/raw/ConnectionTime.csv', low_memory=False)
connection.drop_duplicates(inplace=True)
connection.head()

,dateNum,maxLogOff,Нормативное время начала раб.дня,Фактич. время начала раб.дня,Время опоздания,Признак опоздания,Вых/Будни,id
0,"2021-12-15 00:00:00,000","2021-12-15 17:30:27,246","1899-12-30 08:30:00,000","1899-12-30 08:24:18,606",NaN,NaN,Будни,ОРГ1-02782
1,"2021-12-15 00:00:00,000","2021-12-15 16:30:13,330","1899-12-30 08:30:00,000","1899-12-30 01:05:20,513",NaN,NaN,Будни,ОРГ1-01407
2,"2021-12-15 00:00:00,000","2021-12-15 15:27:37,246","1899-12-30 08:30:00,000","1899-12-30 05:47:33,156",NaN,NaN,Будни,ОРГ1-01909
3,"2021-12-15 00:00:00,000","2021-12-15 20:35:14,833","1899-12-30 08:30:00,000","1899-12-30 08:28:43,956",NaN,NaN,Будни,ОРГ1-00254
4,"2021-12-15 00:00:00,000","2021-12-15 17:32:16,913","1899-12-30 08:30:00,000","1899-12-30 08:16:02,896",NaN,NaN,Будни,ОРГ1-02543


In [30]:
# date column 
connection['date_connected'] = connection['dateNum'].apply(lambda x: x[:10])
# time logoff
connection['time_end_max'] = connection['maxLogOff'].apply(lambda x: x[:19])
# time start normative
connection['time_start_normative'] = connection['Нормативное время начала раб.дня'].apply(lambda x: x[11:19])
# time start fact
connection['time_start_fact'] = connection['Фактич. время начала раб.дня'].apply(lambda x: x[11:19])
# drop unuseful cols
connection.drop(['Нормативное время начала раб.дня', 'Фактич. время начала раб.дня'], axis=1, inplace=True)

In [31]:
connection['date_connected'] = pd.to_datetime(connection['date_connected'], format='%Y-%m-%d')
connection['time_end_max'] = pd.to_datetime(connection['time_end_max'], format='%Y-%m-%d %H:%M:%S')

connection['time_start_normative'] = pd.to_datetime(connection['time_start_normative'], format='%H:%M:%S')
connection['time_start_fact'] = pd.to_datetime(connection['time_start_fact'], format='%H:%M:%S')

In [32]:
min_date = connection['date_connected'].min()
print('date_start_data: ', min_date)
max_date = connection['date_connected'].max()
print('date_end_data: ', max_date)

date_start_data:  2021-06-20 00:00:00
date_end_data:  2021-12-31 00:00:00


In [33]:
for i in connection.index:
    connection['time_start_normative'].loc[i] = dt.combine(connection['date_connected'].loc[i].date(
    ), connection['time_start_normative'].loc[i].time())

/var/folders/dd/p98qrkfn48g4vx43m7wc74rc0000gp/T/ipykernel_13972/3971472013.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  connection['time_start_normative'].loc[i] = dt.combine(connection['date_connected'].loc[i].date(


In [34]:
for i in connection.index:
    connection['time_start_fact'].loc[i] = dt.combine(connection['date_connected'].loc[i].date(
    ), connection['time_start_fact'].loc[i].time())

/var/folders/dd/p98qrkfn48g4vx43m7wc74rc0000gp/T/ipykernel_13972/1687800683.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  connection['time_start_fact'].loc[i] = dt.combine(connection['date_connected'].loc[i].date(


In [35]:
# duplicated date and id
print('Count of duplicated date and id: ', connection[['date_connected','id']].duplicated().sum())

Count of duplicated date and id:  52125


In [36]:
# sort rows by id and time_start_fact ascending
connection = connection.sort_values(['id','time_start_fact'])

# drop duplicates by date and id, keep first (means keep only first login in each day)
connection = connection.drop_duplicates(subset=['id','date_connected'], keep='first')

# duplicated date and id check after clean
print('Count of duplicated date and id: ', connection[['date_connected','id']].duplicated().sum())

Count of duplicated date and id:  0


In [37]:
# time start deviation
connection['time_connected_deviation'] = (connection['time_start_fact'] - connection['time_start_normative']
 ).apply(lambda x: x.total_seconds() / 3600)

In [38]:
# binary is_weekend or is_workday
connection['is_workday_connected'] = np.where(connection['Вых/Будни'] == 'Будни', 1, 0)
connection['is_weekend_connected'] = np.where(connection['Вых/Будни'] == 'Выходные дни', 1, 0)

In [39]:
# drop unnecessary columns
connection = connection.drop(['dateNum', 'maxLogOff', 'Время опоздания','Признак опоздания', 'Вых/Будни'], axis=1)

In [40]:
# is late binary column create (late = came more than 10 minus after start)
connection['is_late_connected'] = np.where(connection['time_connected_deviation'] > 0.17, 1, 0)
# is late more than one hour
connection['late_more_than_hour'] = np.where(connection['time_connected_deviation'] > 1, 1, 0)
# early came to work (earlier than 1 hour)
connection['early_more_than_hour'] = np.where(connection['time_connected_deviation'] < 1, 1 ,0)

In [41]:
# total time connected during session
connection['total_time_connected'] = (connection['time_end_max'] - connection['time_start_fact']).apply(
    lambda x: x.total_seconds() / 3600)

In [42]:
# aggregation function to calculate employment time
def work_period_by_connection(dates):
    if (dates.max() - dates.min()).days == 0:
        return 1
    else:
        return (dates.max() - dates.min()).days

In [43]:
# agg function to find days since last connect
def days_since_last_connection(dates, max_date=max_date):
    return (max_date - dates.max()).days

In [44]:
connection_last_150 = connection[connection['date_connected'] > (max_date - timedelta(days=150))]
connection_last_100 = connection[connection['date_connected'] > (max_date - timedelta(days=100))]
connection_last_50 = connection[connection['date_connected'] > (max_date - timedelta(days=50))]

In [45]:
def aggregation(connection): 
# aggregation connection df
    grouped_connection = connection.groupby('id', as_index=False).agg({
    'date_connected':['count', work_period_by_connection, days_since_last_connection],
    'time_connected_deviation':['sum', 'mean', 'median', 'std', 'min', 'max'],
    'total_time_connected':['sum','mean', 'median', 'std', 'min', 'max'],
    'is_late_connected':['sum','mean'],
    'early_more_than_hour': ['sum','mean'],
    'late_more_than_hour':['sum', 'mean'],
    'is_workday_connected':['sum','mean'],
    'is_weekend_connected':['sum','mean']  
})
    grouped_connection.columns = grouped_connection.columns.map('_'.join).str.strip('_')
    return grouped_connection

In [46]:
grouped_connection = aggregation(connection)

grouped_connection['mean_time_due_employed'] = grouped_connection['total_time_connected_sum'] / \
    grouped_connection['date_connected_work_period_by_connection']
grouped_connection.columns = grouped_connection.columns.map(
    lambda x: x+'_total' if x != 'id' else x)

grouped_connection_last_50 = aggregation(connection_last_50)
grouped_connection_last_50.columns = grouped_connection_last_50.columns.map(
    lambda x: x+'_last_50' if x != 'id' else x)

grouped_connection_last_100 = aggregation(connection_last_100)
grouped_connection_last_100.columns = grouped_connection_last_100.columns.map(
    lambda x: x+'_last_100' if x != 'id' else x)

grouped_connection_last_150 = aggregation(connection_last_150)
grouped_connection_last_150.columns = grouped_connection_last_150.columns.map(
    lambda x: x+'_last_150' if x != 'id' else x)

In [47]:
# merge all dfs in one
connection_dfs_for_merge = [grouped_connection, grouped_connection_last_150,
                            grouped_connection_last_100, grouped_connection_last_50]

grouped_data = reduce(lambda left, right: pd.merge(left, right, on=['id'],
                                                   how='left'), connection_dfs_for_merge)

In [48]:
grouped_data['exists_in_connection'] = 1

In [49]:
grouped_data.to_csv('../data/prepared/grouped_connection.csv', index=False)